In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from Exercise02AI import Exercise02AI

# Aufgabe 03: Minimax (Simplified Evaluation Function)

Dieses Notebook implementiert die Simplified Evaluation Function zur Berechnung des nächsten Zuges im Mittelspiel. Hierbei wird die `minimax`-Funktion der Super-Klasse (`Exercise02AI`) aufgerufen und durch die nun implementierte Simplified Evaluation Function verbessert.

Hierzu werden zu Beginn die in der [Simplified Evaluation Function](https://www.chessprogramming.org/Simplified_Evaluation_Function#Piece-Square_Tables) beschriebenen `Piece-Square Tables` definiert. Diese weisen jeder der 64 möglichen Positionen auf dem Schachfeld für alle der Schachfiguren (Bauer, Springer, Läufer, Turm, Dame und König) eine numerische Bewertung in Form einer Ganzzahl zu. Diese kann sowohl positiv (gute Position für die Figur) oder negativ (schlechte Position für die Figur) sein und wird als Liste dargestellt. Der König verfügt über zwei verschiedene Listen, eine für das Mittelspiel, eine für das Endspiel.

Anhand der Piece-Square Table der Bauern `PST_PAWN`, die im Folgenden definiert ist, wird ersichtlich, dass Bauern auf den Feldern `f2`, `g2` und `h2` (symmetrischer Weise natürlich auch auf `a2`, `b2` und `c2`) Boni (positive Bewertung) erhalten, während sie auf den Felder `f3` und `g3` Abzüge (negative Bewertungen) erhalten.

Das Dictionary `PIECE_TO_PST` weist jedem Figurentyp die entsprechende umgedrehte Piece-Square Table zu, welche aufgrund der einfacheren Lesbarkeit in umgekehrter Reihenfolge initialisiert werden. Um für die Figur des Königs zwischen Mittel- und Endpiel unterscheiden zu können, existiert zusätzlich das Dictionary `PIECE_TO_PST_KING_ENDGAME`.

Wie auch schon in `Exercise02AI` existiert eine Funktion `reset`, die dazu dient, die Instanz nach einem beendeten Spiel in ihren Ursprungszustand zurückzusetzen.

In [ ]:
def mirror_pst(PST: list) -> list:
    """Mirrors a piece square table on the x-axis"""
    rows = [PST[i : i + 8] for i in range(0, len(PST), 8)]
    return [field for row in reversed(rows) for field in row]

In [ ]:
import chess


class Exercise03AI(Exercise02AI):
    """Chooses middle game moves using minimax algorithm and piece square tables."""

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.is_king_endgame: bool = False
        # Create piece square table mapping (needs to revert tables)
        self.PIECE_TO_PST = {
            chess.PAWN: mirror_pst(self.PST_PAWN),
            chess.KNIGHT: mirror_pst(self.PST_KNIGHT),
            chess.BISHOP: mirror_pst(self.PST_BISHOP),
            chess.ROOK: mirror_pst(self.PST_ROOK),
            chess.QUEEN: mirror_pst(self.PST_QUEEN),
            chess.KING: mirror_pst(self.PST_KING),
        }
        self.PIECE_TO_PST_KING_ENDGAME = {
            chess.PAWN: mirror_pst(self.PST_PAWN),
            chess.KNIGHT: mirror_pst(self.PST_KNIGHT),
            chess.BISHOP: mirror_pst(self.PST_BISHOP),
            chess.ROOK: mirror_pst(self.PST_ROOK),
            chess.QUEEN: mirror_pst(self.PST_QUEEN),
            chess.KING: mirror_pst(self.PST_KING_ENDGAME),
        }
        # Start with normal tables
        self.tables = self.PIECE_TO_PST

    def reset(self) -> None:
        """Resets all internal variables."""
        super().reset()
        self.is_king_endgame = False
        self.tables = self.PIECE_TO_PST

    # Define field values as class constants
    # fmt: off
    PST_PAWN = [
        0,  0,  0,  0,  0,  0,  0,  0,
       50, 50, 50, 50, 50, 50, 50, 50,
       10, 10, 20, 30, 30, 20, 10, 10,
        5,  5, 10, 25, 25, 10,  5,  5,
        0,  0,  0, 20, 20,  0,  0,  0,
        5, -5,-10,  0,  0,-10, -5,  5,
        5, 10, 10,-20,-20, 10, 10,  5,
        0,  0,  0,  0,  0,  0,  0,  0
    ]

    PST_KNIGHT = [
        -50,-40,-30,-30,-30,-30,-40,-50,
        -40,-20,  0,  0,  0,  0,-20,-40,
        -30,  0, 10, 15, 15, 10,  0,-30,
        -30,  5, 15, 20, 20, 15,  5,-30,
        -30,  0, 15, 20, 20, 15,  0,-30,
        -30,  5, 10, 15, 15, 10,  5,-30,
        -40,-20,  0,  5,  5,  0,-20,-40,
        -50,-40,-30,-30,-30,-30,-40,-50
    ]

    PST_BISHOP = [
        -20,-10,-10,-10,-10,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5, 10, 10,  5,  0,-10,
        -10,  5,  5, 10, 10,  5,  5,-10,
        -10,  0, 10, 10, 10, 10,  0,-10,
        -10, 10, 10, 10, 10, 10, 10,-10,
        -10,  5,  0,  0,  0,  0,  5,-10,
        -20,-10,-10,-10,-10,-10,-10,-20
    ]

    PST_ROOK = [
        0,  0,  0,  0,  0,  0,  0,  0,
        5, 10, 10, 10, 10, 10, 10,  5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
        0,  0,  0,  5,  5,  0,  0,  0
    ]

    PST_QUEEN = [
        -20,-10,-10, -5, -5,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5,  5,  5,  5,  0,-10,
         -5,  0,  5,  5,  5,  5,  0, -5,
          0,  0,  5,  5,  5,  5,  0, -5,
        -10,  5,  5,  5,  5,  5,  0,-10,
        -10,  0,  5,  0,  0,  0,  0,-10,
        -20,-10,-10, -5, -5,-10,-10,-20
    ]

    PST_KING = [
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -20,-30,-30,-40,-40,-30,-30,-20,
        -10,-20,-20,-20,-20,-20,-20,-10,
         20, 20,  0,  0,  0,  0, 20, 20,
         20, 30, 10,  0,  0, 10, 30, 20
    ]

    PST_KING_ENDGAME = [
        -50,-40,-30,-20,-20,-30,-40,-50,
        -30,-20,-10,  0,  0,-10,-20,-30,
        -30,-10, 20, 30, 30, 20,-10,-30,
        -30,-10, 30, 40, 40, 30,-10,-30,
        -30,-10, 30, 40, 40, 30,-10,-30,
        -30,-10, 20, 30, 30, 20,-10,-30,
        -30,-30,  0,  0,  0,  0,-30,-30,
        -50,-30,-30,-30,-30,-30,-30,-50
    ]
    # fmt: on

## Prüfung der Endspielkonditionen

Um bei der Simplified Evaluation Function die richtige Königstabelle auszuwählen ist es notwendig zu prüfen, ob sich das aktuelle Spiel im Mittel- oder Endspiel befindet. Das Endspiel wird dabei definiert als ein Spiel bei dem entweder:

1. beide Seiten keine Dame mehr besitzen oder
2. jede Seite die eine Dame hat, keinen Turm und maximal einen Läufer oder Springer besitzt.

**Hinweis**: Diese Definition der Endspielkonditionen bezieht sich nur auf die Simplified Evaluation Function. Allgemein ist das Endspiel (implementiert durch die Syzygy Endgame Tablebases) erreicht, sobald 5 oder weniger Figuren auf dem Spielbrett vorhanden sind.

Die Funktion `check_king_endgame` implementiert die oben genannten Bedingungen und prüft, ob sie für ein gegebenes Board erfüllt sind.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    def check_king_endgame(self, board: chess.Board) -> bool:
        """Checks if the king endgame tables should be used."""
        return all(
            (
                (
                    not board.pieces(chess.QUEEN, color)
                    or (
                        len(board.pieces(chess.BISHOP, color))
                        + len(board.pieces(chess.KNIGHT, color))
                        < 2
                    )
                    and not board.pieces(chess.ROOK, color)
                )
                for color in chess.COLORS
            )
        )

## Evaluierungsfunktion

Die Funktion `calculate_pst_value` nimmt als Argument einen Figurentyp (`piece`), eine Farbe (`color`) und eine Menge von Positionen (`positions`). Anschließend wird für jede Position der Wert in der entsprechenden Piece-Square Table ermittelt und die Summe dieser Werte zurückgegeben. Für schwarze Figuren wird das Board dabei gespiegelt und das Resultat negiert.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    def calculate_pst_value(
        self, piece: chess.PieceType, color: chess.Color, positions: chess.SquareSet
    ) -> int:
        """Calculates the piece-square table values for a given piece, color and and optional position."""
        if color == chess.WHITE:
            return sum([self.tables[piece][i] for i in positions])
        else:
            return -sum(
                [(self.tables[piece][chess.square_mirror(i)]) for i in positions]
            )

Die Funktion `evaluate` aus der Elternklasse (`Exercise02AI`) wird unverändert übernommen. Die Funktionen `full_evaluate` und `incremental_evaluate` werden erweitert. Hier werden die Materialwerte der einzelnen Figuren in *Centipawns* als Berechnungsgrundlage benötigt. Folglich werden die bereits in der Elternklasse gesetzten Werte wie folgt angepasst:

| Figurname  | Materialwert |
|---|---|
| Bauer (pawn) | 100  |
| Springer (knight) | 320  |
| Läufer (bishop) | 330  |
| Turm (rook) | 500  |
| Dame (queen) | 900  |
| König (king) | 20000 |

Die folgenden Funktionen der Elternklasse wurden überschrieben:

- `full_evaluate`
- `incremental_evaluate`

Die Funktion `full_evaluate` berechnet den Wert eines Boards von Grund auf neu und geht dabei wie folgt vor:

1. Es wird für jede Figur und für jede Farbe mithilfe der Funktion `calculate_pst_value` die Bewertung der Figur in Centipawns ermittelt.
2. Es wird die Summe über alle Bewertungen gebildet. Je größer die Summe, desto besser ist Weiß auf dem Feld positioniert, umgekehrt ist ein negatives Ergebnis ein Indikator dafür, dass Schwarz besser auf dem Board positioniert ist.
3. Die Summe aller Bewertungen wird auf den Rückgabewert der `full_evaluate` Funktion der Elternklasse addiert und zurückgegeben.

Der Rückgabewert der Funktion beschreibt also die Kombination des reinen Materialwertes der Figuren beider Spieler und der Positionsbewertung durch die `Piece-Square Tables`.

Die Funktion `incremental_evaluate` berechnet die Änderungen der Bewertung, wenn der gegebene Zug (`next_move`) auf das übergebene Board (`board`) angewendet wird. Diese Änderung wird zurückgegeben. Hierbei werden die folgenden Fälle beachtet:

- Umwandlung eines Bauern in eine höherwertige Figur
- Veränderung der Position der gezogenen Figur
- Schlagen einer Figur (auch durch einen En Passant Zug)
- Lange (große) und kurze (kleine) Rochade

Wird ein Bauer in eine höherwertige Figur umgewandelt, so wird die alte Positionsbewertung des gezogenen Bauern von der Bewertung der umgewandelten Figur subtrahiert. Das Ergebnis wird in der Variable `change` gespeichert.
War der letzte Zug keine Umwandlung, so wird die alte Bewertung der gezogenen Figur von ihrer neuen Bewertung abgezogen. Das Ergebnis wird ebenfalls in `change` gespeichert.

Handelt es sich bei dem letzten Zug um eine lange Rochade, so ändert sich die Positionsbewertung des Turms. Hierzu wird der Materialwert eines Turms auf `change` addiert, falls Weiß am Zug ist, oder von `change` subtrahiert, falls Schwarz am Zug ist. Bei einer kurzen Rochade entsteht keine Veränderung der Positionsbewertung durch den bewegten Turm.

Falls während des letzten Zuges eine Figur geschlagen wurde, wird die alte Bewertung der geschlagenen Figur errechnet und von `change` subtrahiert. Hierbei wird eine separate Abfrage verwendet, um zu erkennen, ob es sich bei dem letzten Zug um einen En Passant Zug handelt.

Zurückgegeben wird schlussendlich die Summe aus `change` und dem Rückgabewert der Funktion `incremental_evaluate` der Elternklasse.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    MATERIAL_VALUES = {
        chess.PAWN: 100,
        chess.KNIGHT: 320,
        chess.BISHOP: 330,
        chess.ROOK: 500,
        chess.QUEEN: 900,
        chess.KING: 20000,
    }

    def full_evaluate(self, board: chess.Board) -> int:
        """Evaluates a given board."""
        # Get if pieces standing well or pieces standing badly by calculating with the Piece-Square Tables
        rating = sum(
            self.calculate_pst_value(piece_type, color, board.pieces(piece_type, color))
            for piece_type in chess.PIECE_TYPES
            for color in chess.COLORS
        )
        return rating + super().full_evaluate(board)

    def incremental_evaluate(self, board: chess.Board, next_move: chess.Move) -> int:
        """Returns an incrementally calculated evaluation of the given board."""
        if promotion_piece_type := next_move.promotion:
            # Calculate change caused by piece promotion
            old_piece_pst = self.calculate_pst_value(
                chess.PAWN, board.turn, [next_move.from_square]
            )
            new_piece_pst = self.calculate_pst_value(
                promotion_piece_type, board.turn, [next_move.to_square]
            )
            change = new_piece_pst - old_piece_pst
        else:
            # Calculate change caused by piece move
            piece_type = board.piece_type_at(next_move.from_square)
            old_pst_value = self.calculate_pst_value(
                piece_type, board.turn, [next_move.from_square]
            )
            new_pst_value = self.calculate_pst_value(
                piece_type, board.turn, [next_move.to_square]
            )
            change = new_pst_value - old_pst_value

        if captured_piece_type := board.piece_type_at(next_move.to_square):
            # Calculate additional change caused by piece capture
            captured_piece_pst = self.calculate_pst_value(
                captured_piece_type, not board.turn, [next_move.to_square]
            )
            change -= captured_piece_pst
        elif board.is_en_passant(next_move):
            # Calculate additional change caused by en passant pawn capture
            pawn_double_move = board.peek()
            captured_pawn_pst = self.calculate_pst_value(
                chess.PAWN, not board.turn, [pawn_double_move.to_square]
            )
            change -= captured_pawn_pst
        elif board.is_queenside_castling(next_move):
            # Add changed rook value caused by long castling (short castling does not change rook value and can be ignored)
            change += 5 * (1 if board.turn else -1)

        return change + super().incremental_evaluate(board, next_move)

## Berechnung des besten Zuges

Die Funktion `get_next_middle_game_move` berechnet auf einem gegebenen Board den nächsten besten Zug. Zusätzlich zur bereits vorhandenen Funktionalität der Elternklasse wird hier überprüft, ob sich der Endspielstatus geändert hat. Falls zutreffend, wird die letzte gespeicherte Beurteilung mithilfe der Funktion `get_endgame_evaluation_change` entsprechend angepasst, da der König über eine separate `Piece-Square Table` für das Endspiel verfügt. Aus Performance-Gründen wird der Endspielstatus innerhalb der Minimax Funktion nicht jedes Mal erneut überprüft.

**Hinweis**: `self.last_evaluation` kann in der Theorie den Wert `None` annehmen und hier einen Fehler auslösen. Dies ist aber nur zu Beginn des Mittelspiels der Fall. Der Status des Spiels zum Endspiel ändert sich aber erst im späteren Spielverlauf, daher tritt der Fall in der Praxis nicht ein und wird folglich nicht geprüft.

In [ ]:
class Exercise03AI(Exercise03AI):  # type: ignore
    def get_endgame_evaluation_change(self, board: chess.Board) -> int:
        """Calculates the change of the evaluation when switching endgame status."""
        self.is_king_endgame: bool  # mypy type declaration
        old_rating_king_white = self.calculate_pst_value(
            chess.KING, chess.WHITE, board.pieces(chess.KING, chess.WHITE)
        )
        old_rating_king_black = self.calculate_pst_value(
            chess.KING, chess.BLACK, board.pieces(chess.KING, chess.BLACK)
        )
        self.is_king_endgame = not self.is_king_endgame
        self.tables = (
            self.PIECE_TO_PST_KING_ENDGAME
            if self.is_king_endgame
            else self.PIECE_TO_PST
        )
        new_rating_king_white = self.calculate_pst_value(
            chess.KING, chess.WHITE, board.pieces(chess.KING, chess.WHITE)
        )
        new_rating_king_black = self.calculate_pst_value(
            chess.KING, chess.BLACK, board.pieces(chess.KING, chess.BLACK)
        )
        return (
            new_rating_king_white
            - old_rating_king_white
            + new_rating_king_black
            - old_rating_king_black
        )

    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move."""
        self.last_evaluation: int  # type annotation for mypy
        if self.is_king_endgame != self.check_king_endgame(board):
            self.last_evaluation += self.get_endgame_evaluation_change(board)
        return super().get_next_middle_game_move(board)

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
from AIBaseClass import ChessAI

In [ ]:
# Create player and board
unit_test_player = Exercise03AI(player_name="Ex03AI", search_depth=2)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
next_move = chess.Move.from_uci("d4c3")  # white queen capture
board

In [ ]:
# Test endgame check
def test_endgame_check(unit_test_player: ChessAI, board: chess.Board):
    is_endgame_1 = unit_test_player.check_king_endgame(board)
    assert is_endgame_1 is False, "Endgame status should be False!"

    board.push(next_move)
    is_endgame_2 = unit_test_player.check_king_endgame(board)
    assert is_endgame_2 is True, "Endgame status should be True!"
    board.pop()
    print(f"Endgame 1: {is_endgame_1} - Endgame 2: {is_endgame_2}")

In [ ]:
test_endgame_check(unit_test_player, board)

In [ ]:
# Test pst calculation
def test_pst_calculation(unit_test_player: ChessAI, board: chess.Board):
    pst_value_white = unit_test_player.calculate_pst_value(
        piece=chess.PAWN, color=chess.WHITE, positions=[chess.F2]
    )
    assert (
        pst_value_white == 10
    ), "PST calculation for white does not match expected value!"

    pst_value_black = unit_test_player.calculate_pst_value(
        piece=chess.BISHOP, color=chess.BLACK, positions=[chess.B7]
    )
    assert (
        pst_value_black == -5
    ), "PST calculation for black does not match expected value!"
    print(f"PST Values: White {pst_value_white} - Black {pst_value_black}")

In [ ]:
test_pst_calculation(unit_test_player, board)

In [ ]:
# Test full evaluation
def test_full_evaluation(unit_test_player: ChessAI, board: chess.Board):
    full_eval = unit_test_player.full_evaluate(board)
    assert full_eval == 1240, "Full evaluation does not match expected value!"
    print(f"Full Evaluation: {full_eval}")

In [ ]:
test_full_evaluation(unit_test_player, board)

In [ ]:
# Test incremental evaluation
def test_incremental_evaluation(unit_test_player: ChessAI, board: chess.Board):
    next_move = chess.Move.from_uci("d4c3")  # white queen capture
    inc_eval = unit_test_player.incremental_evaluate(board, next_move)
    assert inc_eval == -900, "Incremental evaluation does not match expected value!"
    print(f"Incremental Evaluation: {inc_eval}")

In [ ]:
test_incremental_evaluation(unit_test_player, board)

In [ ]:
# Test evaluation
def test_evaluation(unit_test_player: ChessAI, board: chess.Board):
    board.push(next_move)
    full_eval, inc_eval = 1240, -900
    evaluation = unit_test_player.evaluate(board, last_evaluation=full_eval)
    assert evaluation == full_eval + inc_eval, "Evaluation does not match expected sum!"
    new_full_eval = unit_test_player.full_evaluate(board)
    assert new_full_eval == evaluation, "Incremental and full evaluation are different!"
    board.pop()
    print(f"Full Evaluation: {full_eval}")
    print(f"Incremental Evaluation: {inc_eval}")
    print(f"Evaluation: {evaluation}")
    print(f"New Full Evaluation: {new_full_eval}")

In [ ]:
test_evaluation(unit_test_player, board)

In [ ]:
# Test endgame change calculation
def test_endgame_change_calc(unit_test_player: ChessAI, board: chess.Board):
    change_to_endgame = unit_test_player.get_endgame_evaluation_change(board)
    assert (
        change_to_endgame == 10
    ), "Endgame change calculation does not match expected value!"

    change_from_endgame = unit_test_player.get_endgame_evaluation_change(board)
    assert (
        change_from_endgame == -10
    ), "Endgame change calculation does not match expected value!"
    print(f"Change to endgame: {change_to_endgame}")
    print(f"Change from endgame: {change_from_endgame}")

In [ ]:
test_endgame_change_calc(unit_test_player, board)

In [ ]:
# Test minimax
def test_minimax(unit_test_player: ChessAI, board: chess.Board):
    mm_evaluation, mm_move = unit_test_player.minimax(
        board, depth=2, current_evaluation=1240
    )
    assert mm_evaluation == 355, "Minimax evaluation does not match expected value!"
    assert mm_move.uci() == "d4c3", "Minimax move does not match expected value!"
    print(f"Minimax Evaluation: {mm_evaluation}")
    print(f"Minimax Move: {mm_move}")

In [ ]:
test_minimax(unit_test_player, board)

In [ ]:
def test_next_move(unit_test_player: ChessAI, board: chess.Board):
    move = unit_test_player.get_next_middle_game_move(board)
    assert move.uci() == "d4c3", "Next move does not match expected value!"
    print(f"Move: {move}")

In [ ]:
test_next_move(unit_test_player, board)

In [ ]:
# Test reset function
def test_reset(unit_test_player: ChessAI, _: chess.Board):
    unit_test_player.reset()
    assert (
        unit_test_player.last_evaluation is None
    ), "Reset was not successful! (last_evaluation)"
    assert (
        unit_test_player.is_king_endgame is False
    ), "Reset was not successful! (is_king_endgame)"
    assert (
        unit_test_player.tables == unit_test_player.PIECE_TO_PST
    ), "Reset was not successful! (tables)"

In [ ]:
test_reset(unit_test_player, board)

In [ ]:
def test_pst(unit_test_player: ChessAI, board: chess.Board, expected_value: int):
    rating = sum(
        unit_test_player.calculate_pst_value(
            piece_type, color, board.pieces(piece_type, color)
        )
        for piece_type in chess.PIECE_TYPES
        for color in chess.COLORS
    )
    print(f"PST evaluation: {rating}")
    assert (
        rating == expected_value
    ), f"PST evaluation does not match expected value of {expected_value}!"

In [ ]:
board_pst = chess.Board("4k3/8/8/8/Q7/8/8/4K3 w - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=0)

In [ ]:
board_pst = chess.Board("4k3/8/8/8/7Q/8/8/4K3 w - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=-5)

In [ ]:
board_pst = chess.Board("4k3/8/8/7Q/8/8/8/4K3 b - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=-5)

In [ ]:
board_pst = chess.Board("4k3/8/8/q7/8/8/8/4K3 w - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=0)

In [ ]:
board_pst = chess.Board("4k3/8/8/7q/8/8/8/4K3 w - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=5)

In [ ]:
board_pst = chess.Board("4k3/8/7q/8/8/8/8/4K3 w - - 0 1")
# display(board_pst)
test_pst(unit_test_player, board_pst, expected_value=10)

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.